In [1]:
import pandas as pd

data = pd.read_csv('aseinio-modiwlau.csv')

In [2]:
data

,Unnamed: 0,ID,Marc Cymedrig Blwyddyn 1,Modiwl 1,Modiwl 10,Modiwl 11,Modiwl 12,Modiwl 13,Modiwl 14,Modiwl 15,...,Modiwl 20,Modiwl 3,Modiwl 4,Modiwl 5,Modiwl 6,Modiwl 7,Modiwl 8,Modiwl 9,Presenoldeb Cymedrig Blwyddyn 1,Rhyw
0,0,0,51,0,0,3,0,6,0,0,...,0,0,0,4,1,7,0,5,95.7,F
1,1,1,71,5,0,0,0,0,4,0,...,2,6,0,7,0,0,0,0,86.8,F
2,2,2,64,0,0,0,0,3,1,0,...,0,6,2,4,0,7,0,0,82.6,M
3,3,3,98,0,0,0,0,0,5,0,...,3,1,0,0,4,2,0,0,75.4,F
4,4,4,55,5,0,4,0,0,0,0,...,0,0,0,0,0,6,0,0,78.0,F
5,5,5,86,0,0,7,0,2,0,0,...,0,0,0,5,4,0,0,6,93.1,F
6,6,6,85,3,7,0,4,0,0,0,...,5,0,2,0,0,0,0,0,99.1,F
7,7,7,91,6,3,7,0,0,5,0,...,0,0,4,0,0,0,0,1,99.4,F
8,8,8,87,2,0,0,0,0,7,3,...,4,0,0,0,5,0,0,6,81.5,F
9,9,9,68,0,0,0,6,2,0,0,...,0,0,0,3,5,0,0,0,53.4,M


In [3]:
import pulp

In [4]:
prob = pulp.LpProblem("Aseinio Modiwlau", pulp.LpMaximize)

N_myfyrwyr = len(data['ID'])
D = pulp.LpVariable.dicts("D", [(i, j) for i in range(N_myfyrwyr) for j in range(20)], cat=pulp.LpBinary)

for i in range(N_myfyrwyr):
    prob += sum([D[(i, j)] for j in range(20)]) == 3
    
for j in range(20):
    prob += sum([D[(i, j)] for i in range(N_myfyrwyr)]) <= 40
    prob += sum([D[(i, j)] for i in range(N_myfyrwyr)]) >= 6

In [5]:
# f1: Y ffwythiant amcan gwreiddiol.

f1 = sum([data['Modiwl ' + str(j + 1)][i] * D[(i, j)] for i in range(N_myfyrwyr) for j in range(20)])

In [6]:
# f2: Yn ddelfrydol dylai fod 30 myfyrwyr ar bob modiwl.

F2 = pulp.LpVariable.dicts("F2", [i for i in range(20)]) # rhestr o newidynnau dewis i adeiladu f2

for j in range(20):
    prob += F2[j] >= 30 - sum([D[(i, j)] for i in range(N_myfyrwyr)])
    prob += F2[j] >= sum([D[(i, j)] for i in range(N_myfyrwyr)]) - 30
    
f2 = -sum([F2[j] for j in range(20)])

In [7]:
# f3: Yn ddelfrydol dylai nifer o fyfyrwyr sy’n mynychu gwersi (yn seiliedig ar bresenoldeb y flwyddyn cynt) fod tua 30.

disg_n = [sum([data['Presenoldeb Cymedrig Blwyddyn 1'][i] * D[(i, j)] /100 for i in range(N_myfyrwyr)]) for j in range(20)]
# rhestr o gwerthoedd disgwyledig o'r nifer of myfyrwir fydd yn mynychu dosbarth

F3 = pulp.LpVariable.dicts("F3", [i for i in range(20)])

for j in range(20):
    prob += F3[j] >= 30 - disg_n[j]
    prob += F3[j] >= disg_n[j] - 30
    
f3 = -sum([F3[j] for j in range(20)])

In [8]:
# f4: Yn ddelfrydol dylai maint dosbarthiadau bod yn hafal

F4 = pulp.LpVariable.dicts("F4", [i for i in range(20)])

mu = N_myfyrwyr/20 # cymedr nifer o myfyrwir ym mhob ddosbarth
for j in range(20):
    prob += F4[j] >= mu - sum([D[(i, j)] for i in range(N_myfyrwyr)])
    prob += F4[j] >= sum([D[(i, j)] for i in range(N_myfyrwyr)]) - mu
    
f4 = -sum([F4[j] for j in range(20)])

In [9]:
# f5: Yn ddelfrydol dylai fod dosraniad y marciau disgwyledig (yn seiliedig ar farciau blwyddin ddiwethaf) bod yn hafal ar draws modiwlau.

disg_swm_marciau = [sum([data['Marc Cymedrig Blwyddyn 1'][i] * D[(i, j)] for i in range(N_myfyrwyr)]) for j in range(20)]
# rhestr o gwerthoedd disgwyledig o'r swm o farciau gath y dosbarth

F5 = pulp.LpVariable.dicts("F5", [i for i in range(20)])

for j in range(20):
    prob += F5[j] >= 30 - disg_swm_marciau[j]
    prob += F5[j] >= disg_swm_marciau[j] - 30
    
f5 = -sum([F5[j] for j in range(20)])

In [10]:
# f6: Yn ddelfrydol dylai myfyrywyr da (yn seiliedig ar farciau blwyddyn ddiwethaf) bod yn fwy tebygol o gael ei dewisiadau cyntaf.

f6 = sum([data['Modiwl ' + str(j + 1)][i] * D[(i, j)] * data['Marc Cymedrig Blwyddyn 1'][i] for i in range(N_myfyrwyr) for j in range(20)])

In [11]:
def datrys(p1=1, p2=0, p3=0, p4=0, p5=0, p6=0, prob=prob):
    f = p1*(f1 - 6*N_myfyrwyr)/(12*N_myfyrwyr) + p2*f2/(24*20) + p3*f3/(24*20) + p4*f4/(min(mu-6, 40-mu)*20) + p5*f5/(24*20) + p6*f6/(1800*N_myfyrwyr)
    prob += f
    prob.solve()

In [12]:
datrys(1, 1, 1, 1, 1, 1)

In [13]:
datrysiad = [[D[(i, j)].value() for j in range (20)] for i in range(N_myfyrwyr)]
datrysiad

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0],
 [1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0],
 [1.0,
  0.0,